# SESSION_2

**We will use the same env curso_1 from session_1**

Prerequisites: **In a terminal**, You need to create, install biopython and activate the `Conda` env as follow before to start jupyter notebook

!conda install -y -n curso_1 -c bioconda seqtk minimap2 nanoq

!conda activate curso_1

!jupyter notebook &

Download the data [bs_reads.fastq.gz](https://filesender.renater.fr/?s=download&token=475acb28-fcd4-4986-ab82-84733a1f7f5f) before and put the file in the **Session_2/data/bacillus_subtilis** directory. 

# Quality control of fastq reads

How many bases have been sequenced and basecalled ? Use seqtk.  
[Seqtk](https://github.com/lh3/seqtk) is a fast and lightweight tool for processing sequences in the FASTA or FASTQ format. It seamlessly parses both FASTA and FASTQ files which can also be optionally compressed by gzip 

In [3]:
!seqtk seq -A data/bacillus_subtilis/bs_reads.fastq.gz | grep -v ">" |wc -m

 178259749


 Quality Control of FASTQ with [nanoq](https://www.theoj.org/joss-papers/joss.02991/10.21105.joss.02991.pdf)

In [4]:
!nanoq -h

nanoq 0.9.0
Read filters and summary reports for nanopore data

USAGE:
    nanoq [FLAGS] [OPTIONS]

FLAGS:
    -f, --fast       Ignore quality values if present
    -h, --help       Prints help information
    -H, --header     Header for summary output
    -j, --json       Summary report in JSON format
    -s, --stats      Summary report only [stdout]
    -V, --version    Prints version information
    -v, --verbose    Verbose output statistics [multiple, up to -vvv]

OPTIONS:
    -c, --compress-level <1-9>     Compression level to use if compressing output [default: 6]
    -i, --input <input>            Fast{a,q}.{gz,xz,bz}, stdin if not present
    -m, --max-len <INT>            Maximum read length filter (bp) [default: 0]
    -w, --max-qual <FLOAT>         Maximum average read quality filter (Q) [default: 0]
    -l, --min-len <INT>            Minimum read length filter (bp) [default: 0]
    -q, --min-qual <FLOAT>         Minimum average read quality filter (Q) [default: 0]
    -o, -

In [5]:
!nanoq -i data/bacillus_subtilis/bs_reads.fastq.gz -r out_nanoq.txt -s -H

In [ ]:
!cat out_nanoq.txt 

The quality control can also be done by [fastqc](https://timkahlke.github.io/LongRead_tutorials/QC_F.html)  
**Don't do it, Very slow !**  
But I did it for you -> **[HERE](http://localhost:8888/view/Documents/GitHub/Botucatu_Course_March_2024/Session_2/bs_reads_fastqc.html)**

Quality Control of FASTQ with [Nanoplot](https://github.com/wdecoster/NanoPlot)

Install Nanoplot using pip

In [ ]:
!pip install nanoplot

In [ ]:
!NanoPlot --help

Use NanoPlot to ckeck quality of reads (See report.html)

In [ ]:
!NanoPlot -t 1 --fastq data/bacillus_subtilis/bs_reads.fastq --outdir NANOPLOT

Other usefull tools (in https://github.com/wdecoster/NanoPlot).  
`NanoComp`: comparing multiple runs.  
`NanoStat`: statistic summary report of reads or alignments.    
`NanoFilt`: filtering and trimming of reads.  
`NanoLyse`: removing contaminant reads.  
[`FiltLong`](https://github.com/rrwick/Filtlong): filtering long reads by quality 

________________________________________________________________________________________

# MAPPING READS AGAINST A REFERENCE USING MINIMAP2

Installation of [`minimap2`](https://github.com/lh3/minimap2/) with conda before to run Jupyter notebook.  
We have prepared a bigger Bacillus subtilis dataset that was basecalled beforehand. We will try to assemble the sequenced genome. But let's try to use `minimap2` to map fastq reads against the reference to handle files. 
At the next session we will use `minimap2` as the first step of an assembly.

**What to do with Minimap2 ?**

-Map long noisy genomic reads.  
-Map long mRNA/cDNA reads.  
-Find overlaps between long reads.  
-Map short accurate genomic reads.  
-Full genome/assembly alignment.

In [7]:
!minimap2


Usage: minimap2 [options] <target.fa>|<target.idx> [query.fa] [...]
Options:
  Indexing:
    -H           use homopolymer-compressed k-mer (preferrable for PacBio)
    -k INT       k-mer size (no larger than 28) [15]
    -w INT       minimizer window size [10]
    -I NUM       split index for every ~NUM input bases [4G]
    -d FILE      dump index to FILE []
  Mapping:
    -f FLOAT     filter out top FLOAT fraction of repetitive minimizers [0.0002]
    -g NUM       stop chain enlongation if there are no minimizers in INT-bp [5000]
    -G NUM       max intron length (effective with -xsplice; changing -r) [200k]
    -F NUM       max fragment length (effective with -xsr or in the fragment mode) [800]
    -r NUM[,NUM] chaining/alignment bandwidth and long-join bandwidth [500,20000]
    -n INT       minimal number of minimizers on a chain [3]
    -m INT       minimal chaining score (matching bases minus log gap penalty) [40]
    -X           skip self and dual mappings (for the all-vs-all m

We will go ahead and map all our reads to the reference genome without looking for alignments at this point. There are several reasons why we would to this and one of them is to check how many reads are mappable to the reference, and calculate the average base coverage of the sequencing run by choosing the best overlap for each read.

In [ ]:
!minimap2 \
    -x map-ont \
    -o bs_reads_to_ref.paf \
    data/bacillus_subtilis/bs_ref.fasta data/bacillus_subtilis/bs_reads.fastq.gz

#-x map-ont; for Oxford Nanopore reads 

The results is in [PAF (Pairwise mApping Format) format](https://github.com/lh3/miniasm/blob/master/PAF.md).  

See below the tabular format:

|Col|Type|Description  |
|---|---|---|
|1|string|Query sequence name  |
|2|int|Query sequence length  |
|3|int|Query start (0-based; BED-like; closed) | 
|4|int|Query end (0-based; BED-like; open)  |
|5|char|Relative strand: "+" or "-"  |
|6|string|Target sequence name  |
|7|int|Target sequence length  |
|8|int|Target start on original strand (0-based)  |
|9|int|Target end on original strand (0-based)  |
|10|int|Number of residue matches  |
|11|int|Alignment block length  |
|12|int|Mapping quality (0-255; 255 for missing)  |

In [ ]:
!cat bs_reads_to_ref.paf

In [ ]:
import pandas

paf = pandas.read_csv('bs_reads_to_ref.paf', header=None, delimiter='\t')

In [ ]:
paf

In [ ]:
!gunzip -k data/bacillus_subtilis/bs_reads.fastq.gz

In [ ]:
from Bio import SeqIO

reads = []
with open('data/bacillus_subtilis/bs_reads.fastq') as handle:
    for read in SeqIO.parse(handle, 'fastq'):
        reads.append(read)

In [ ]:
print("Num seq        =", len(reads))
print("Num mapped seq =", len(paf[0].unique()))

In [ ]:
paf[paf.duplicated(subset = 0, keep = 'first')]

In [ ]:
paf.drop_duplicates(subset = 0, keep = 'first', inplace = True)
paf.shape

In [ ]:
from matplotlib import pyplot
import seaborn

reflen = paf[6][0]

b = [0] * reflen
e = [0] * reflen
for i, row in paf.iterrows():
    b[row[7]] += 1
    e[row[8]] += 1

pile = [0] * reflen
coverage = 0
for i in range(reflen):
    coverage += b[i]
    pile[i] += coverage
    coverage -= e[i]

print("Avg base coverage =", sum(pile) / reflen)
_ = pyplot.plot(pile)

In [ ]:
_ = pyplot.plot(pile[500000:1000000])

`Minimap` can also find alignments which we will use to calculate the average accuracy of our data. 
This can be done by passing parameter `-a` which will print the output in SAM format, but we can also use `-c` to let minimap2 put the CIGAR string in the last column of the PAF file.

The same thing does ratlesnake, it uses minimizers and the minimap algorithm to find best positions of each read in the reference, and then calculates edit distance with edlib (instead of ksw2 in minimap2). To get the accuracy histogram, we run ratlesnake as in session 1.

In [ ]:
!minimap2

In [ ]:
!minimap2 \
    -x map-ont \
    -c \
    --eqx \
    -o bs_reads_to_ref.cigar.paf \
    data/bacillus_subtilis/bs_ref.fasta \
    data/bacillus_subtilis/bs_reads.fastq.gz
#-c output CIGAR in PAF
#--eqx write =/X CIGAR operators

Install **again** Ratlesnake

In [ ]:
!git clone https://github.com/lbcb-sci/ratlesnake
!mkdir ratlesnake/build
!cmake -S ratlesnake -B ratlesnake/build -DCMAKE_BUILD_TYPE=Release
!make -C ratlesnake/build

In [ ]:
!ratlesnake/build/bin/ratlesnake

In [ ]:
!ratlesnake/build/bin/ratlesnake \
    -r 1 \
    -t 4 \
    data/bacillus_subtilis/bs_reads.fastq.gz \
    data/bacillus_subtilis/bs_ref.fasta

**For Advanced users/players only :**  

In [ ]:
paf = pandas.read_csv('bs_reads_to_ref.cigar.paf', header = None, delimiter = '\t', error_bad_lines = False)
paf.drop_duplicates(subset = 0, keep = 'first', inplace = True)
paf

In [ ]:
def base_accuracy(paf):
    stats = {'=': [], 'X': [], 'I': [], 'D': []}
    for i, row in paf.iterrows():
        if pandas.isnull(row[22]):
            continue
        read = {'=': 0., 'X': 0., 'I': 0., 'D': 0.}
        cigar = row[22][5:]
        n = ''
        for c in cigar:
            if c in ['=', 'X', 'I', 'D']:
                read[c] += int(n)
                n = ''
            else:
                n += c
        for m, n in read.items():
            stats[m].append(n / sum(read.values()))

    return stats

stats = base_accuracy(paf)

In [ ]:
import statistics

def print_statistics(stats):
    print('  Min      Median   Mean     Max')
    for m, n in stats.items():
        print(m, '{0:.6f}'.format(min(n)), '{0:.6f}'.format(statistics.median(n)), '{0:.6f}'.format(statistics.mean(n)), '{0:.6f}'.format(max(n)))

print_statistics(stats)